# Distances API Route Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [ ]:
pip install folium

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

## Get the route between 2 ports

Create a connections towards distances API.
Then call the distance API to get the actual route between Basrah and Milazzo for a Suezmax vessel (corresponding restrictions applied)

In [2]:
from signal_ocean import Connection
from signal_ocean.distances import DistancesAPI, VesselClassFilter, PortFilter, LoadingCondition

connection = Connection(api_key=signal_ocean_api_key)
distances_api = DistancesAPI(connection)

load_port = distances_api.get_ports(PortFilter(name_like='Basrah'))[0]
discharge_port = distances_api.get_ports(PortFilter(name_like='Milazzo'))[0]

vessel_class = distances_api.get_vessel_classes(VesselClassFilter(name_like='suezmax'))[0]
routeResponse = distances_api.get_port_to_port_route(vessel_class, LoadingCondition.BALLAST, load_port, discharge_port)

## Display some properties for the route
The route response objects holds information on properties regarding the route like:
- The total distance covered.
- The seca distance covered within the route.
- The hra distance covered within the route.
- Any significant routing points met across the route (i.e: Suez)

In [3]:

print(f"Total distance: {routeResponse.distance}")
print(f"Seca distance: {routeResponse.seca_distance}")
print(f"Hra distance: {routeResponse.piracy_distance}")
print(f"Significant routing points: {[(routing_point.name) for routing_point in routeResponse.routing_points_on_route if routing_point.is_shown]}")


Total distance: 4574.86
Seca distance: 0
Hra distance: 1301.22
Significant routing points: ['Suez']


## Display the route on a map

Finally you can get all route coordinates and display them on the map.

In [4]:
import folium

coordinatesList = []
for point in routeResponse.calculated_route:
    newList = [float(point.lat), float(point.lon)]
    coordinatesList.append(newList)
m = folium.Map(zoom_start=10)
gh_fg = folium.FeatureGroup(name=f"Path ({load_port.name}) -> ({discharge_port.name})").add_to(m)
folium.PolyLine(locations=coordinatesList, weight=2, color='cadetblue').add_to(gh_fg)
folium.LayerControl().add_to(m)

#Get bounding box of route:
bbox = [[float(routeResponse.bbox[1]), float(routeResponse.bbox[0])], [float(routeResponse.bbox[3]), float(routeResponse.bbox[2])]]
m.fit_bounds(bbox)

#put some markers to display start and stop
folium.Marker(
    location=[routeResponse.start_point.lat, routeResponse.start_point.lon],
    popup= load_port.name,
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[routeResponse.end_point.lat, routeResponse.end_point.lon],
    popup= discharge_port.name,
    icon=folium.Icon(color='red')
).add_to(m)

m